<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/IEEE%20paper_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autokeras


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.utils import get_custom_objects
from keras.layers import Activation
from keras.optimizers import Adam
import keras.backend as K
import autokeras as ak
import time


# Load Pima Indians Diabetes dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)

# Separate features and labels
X = data.drop('class', axis=1).values
Y = data['class'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

def define_activation_functions():
    activations = [
        ('X_1', lambda x: K.pow(x, 1)),
        ('X_2', lambda x: K.pow(x, 2) / 2),
        ('X_2_', lambda x: K.pow(x, 2) / 24),
        ('X_2__', lambda x: K.pow(x, 2) / 720),
        ('X_2___', lambda x: K.pow(x, 2) / 40320),
    ]
    for name, func in activations:
        get_custom_objects().update({name: Activation(func)})

define_activation_functions()

def create_optimized_model(input_dim, hidden_dim, output_dim):
    input_layer = Input(shape=(input_dim,))
    x1 = Dense(hidden_dim, activation='X_1')(input_layer)
    x2 = Dense(hidden_dim, activation='X_2')(input_layer)
    merged1 = concatenate([x1, x2])
    x3 = Dense(hidden_dim, activation='X_2_')(merged1)
    x4 = Dense(hidden_dim, activation='X_2__')(x3)
    x5 = Dense(hidden_dim, activation='X_2___')(x4)
    merged2 = concatenate([x1, x2, x3, x4, x5])
    output = Dense(output_dim, activation='X_1')(merged2)
    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='MeanSquaredError', optimizer=Adam(0.01), metrics=['accuracy'])
    return model

input_dim = 8
hidden_dim = 50
output_dim = 1
n_folds = 10
kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=7)



def run_SWAG_cross_validation(X, Y):
    cvscores = []
    for train, test in kfold.split(X, Y):
        model = create_optimized_model(input_dim, hidden_dim, output_dim)
        model.fit(X[train], Y[train], epochs=5, batch_size=32, verbose=0)
        scores = model.evaluate(X[test], Y[test], verbose=0)
        print(f"Fold Accuracy: {scores[1]*100:.2f}%")
        cvscores.append(scores[1] * 100)
    return cvscores

def run_autokeras_cross_validation(X, Y):
    cvscores = []
    for train, test in kfold.split(X, Y):
        clf = ak.StructuredDataClassifier(overwrite=True, max_trials=5)
        clf.fit(X[train], Y[train], epochs=5, verbose=0)
        scores = clf.evaluate(X[test], Y[test], verbose=0)
        print(f"Fold Accuracy: {scores[1]*100:.2f}%")
        cvscores.append(scores[1] * 100)
    return cvscores

start_time = time.time()

print("SWAG Results")
cvscores_swag = run_SWAG_cross_validation(X, Y)
print(f"{n_folds}-fold cross-validation average accuracy: {np.mean(cvscores_swag):.2f}% (+/- {np.std(cvscores_swag):.2f}%)")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Runtime SWAG : {elapsed_time:.2f} seconds")

start_time = time.time()

print("\nAuto keras Results")
cvscores_ak = run_autokeras_cross_validation(X, Y)
print(f"{n_folds}-fold cross-validation average accuracy: {np.mean(cvscores_ak):.2f}% (+/- {np.std(cvscores_ak):.2f}%)")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Runtime Auto keras : {elapsed_time:.2f} seconds")


SWAG Results
Fold Accuracy: 74.03%
Fold Accuracy: 72.73%
Fold Accuracy: 77.92%
Fold Accuracy: 80.52%
Fold Accuracy: 74.03%
Fold Accuracy: 71.43%
Fold Accuracy: 76.62%
Fold Accuracy: 76.62%
Fold Accuracy: 77.63%
Fold Accuracy: 73.68%
10-fold cross-validation average accuracy: 75.52% (+/- 2.65%)
Runtime SWAG : 29.79 seconds

Auto keras Results
Fold Accuracy: 75.32%
Fold Accuracy: 76.62%
Fold Accuracy: 77.92%
Fold Accuracy: 77.92%
Fold Accuracy: 70.13%
Fold Accuracy: 74.03%
Fold Accuracy: 75.32%
Fold Accuracy: 71.43%
Fold Accuracy: 76.32%
Fold Accuracy: 81.58%
10-fold cross-validation average accuracy: 75.66% (+/- 3.13%)
Runtime Auto keras : 309.03 seconds


In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.utils import get_custom_objects
from keras.layers import Activation
from keras.optimizers import Adam
import keras.backend as K
import autokeras as ak

# Load Ionosphere dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data"
data = pd.read_csv(url, header=None)

# The dataset has 34 features and the last column is the target variable
X = data.drop(34, axis=1).values
# Convert target variable 'g' and 'b' to 1 and 0 respectively
Y = np.where(data[34] == 'g', 1, 0)

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)


input_dim = 34  # There are 34 features in the Ionosphere dataset
hidden_dim = 50
output_dim = 1
n_folds = 10
kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=7)

start_time = time.time()

print("SWAG Results")
cvscores_swag = run_SWAG_cross_validation(X, Y)
print(f"{n_folds}-fold cross-validation average accuracy: {np.mean(cvscores_swag):.2f}% (+/- {np.std(cvscores_swag):.2f}%)")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Runtime SWAG : {elapsed_time:.2f} seconds")

start_time = time.time()

print("\nAuto keras Results")
cvscores_ak = run_autokeras_cross_validation(X, Y)
print(f"{n_folds}-fold cross-validation average accuracy: {np.mean(cvscores_ak):.2f}% (+/- {np.std(cvscores_ak):.2f}%)")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Runtime Auto keras : {elapsed_time:.2f} seconds")


SWAG Results
Fold Accuracy: 91.67%
Fold Accuracy: 82.86%
Fold Accuracy: 82.86%
Fold Accuracy: 94.29%
Fold Accuracy: 97.14%
Fold Accuracy: 91.43%
Fold Accuracy: 85.71%
Fold Accuracy: 97.14%
Fold Accuracy: 80.00%
Fold Accuracy: 94.29%
10-fold cross-validation average accuracy: 89.74% (+/- 6.03%)
Runtime SWAG : 25.61 seconds

Auto keras Results
Fold Accuracy: 94.44%
Fold Accuracy: 68.57%
Fold Accuracy: 71.43%
Fold Accuracy: 42.86%
Fold Accuracy: 91.43%
Fold Accuracy: 85.71%
Fold Accuracy: 80.00%
Fold Accuracy: 88.57%
Fold Accuracy: 82.86%
Fold Accuracy: 77.14%
10-fold cross-validation average accuracy: 78.30% (+/- 14.20%)
Runtime Auto keras : 363.57 seconds


In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.utils import get_custom_objects
from keras.layers import Activation
from keras.optimizers import Adam
import keras.backend as K
import autokeras as ak

# Load the Connectionist Bench (Sonar) dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
data = pd.read_csv(url, header=None)

# The dataset has 60 features and the last column is the target variable
X = data.drop(60, axis=1).values
# Convert target variable 'R' and 'M' to 1 and 0 respectively
Y = np.where(data[60] == 'R', 1, 0)

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

input_dim = 60  # There are 60 features in the Connectionist Bench dataset
hidden_dim = 50
output_dim = 1
n_folds = 10
kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=7)


start_time = time.time()

print("SWAG Results")
cvscores_swag = run_SWAG_cross_validation(X, Y)
print(f"{n_folds}-fold cross-validation average accuracy: {np.mean(cvscores_swag):.2f}% (+/- {np.std(cvscores_swag):.2f}%)")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Runtime SWAG : {elapsed_time:.2f} seconds")

start_time = time.time()

print("\nAuto keras Results")
cvscores_ak = run_autokeras_cross_validation(X, Y)
print(f"{n_folds}-fold cross-validation average accuracy: {np.mean(cvscores_ak):.2f}% (+/- {np.std(cvscores_ak):.2f}%)")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Runtime Auto keras : {elapsed_time:.2f} seconds")


SWAG Results
Fold Accuracy: 80.95%
Fold Accuracy: 66.67%
Fold Accuracy: 76.19%
Fold Accuracy: 85.71%
Fold Accuracy: 85.71%
Fold Accuracy: 71.43%
Fold Accuracy: 90.48%
Fold Accuracy: 80.95%
Fold Accuracy: 80.00%
Fold Accuracy: 70.00%
10-fold cross-validation average accuracy: 78.81% (+/- 7.27%)
Runtime SWAG : 21.71 seconds

Auto keras Results
Fold Accuracy: 52.38%
Fold Accuracy: 57.14%
Fold Accuracy: 42.86%
Fold Accuracy: 66.67%
Fold Accuracy: 47.62%
Fold Accuracy: 57.14%
Fold Accuracy: 52.38%
Fold Accuracy: 66.67%
Fold Accuracy: 65.00%
Fold Accuracy: 55.00%
10-fold cross-validation average accuracy: 56.29% (+/- 7.62%)
Runtime Auto keras : 388.83 seconds


In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
import keras.backend as K

# Custom activation function
def squared_activation(x):
    return K.square(x)

# Load Ionosphere dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data"
data = pd.read_csv(url, header=None)

# The dataset has 34 features and the last column is the target variable
X = data.drop(34, axis=1).values
# Convert target variable 'g' and 'b' to 1 and 0 respectively
Y = np.where(data[34] == 'g', 1, 0)

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build the neural network with deeper architecture
input_layer = Input(shape=(34,))
hidden_layer1 = Dense(50, activation=squared_activation)(input_layer)
hidden_layer2 = Dense(50, activation=squared_activation)(hidden_layer1)
hidden_layer3 = Dense(50, activation=squared_activation)(hidden_layer2)
hidden_layer4 = Dense(50, activation=squared_activation)(hidden_layer3)
output_layer = Dense(1, activation='linear')(hidden_layer4)  # Linear activation for the output layer

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/50
9/9 [==============================] - 1s 27ms/step - loss: 6194360483840.0000 - accuracy: 0.3857 - val_loss: 2140515467264.0000 - val_accuracy: 0.3944
Epoch 2/50
9/9 [==============================] - 0s 8ms/step - loss: 6518625796096.0000 - accuracy: 0.3857 - val_loss: 1018563657728.0000 - val_accuracy: 0.3803
Epoch 3/50
9/9 [==============================] - 0s 9ms/step - loss: 327897317376.0000 - accuracy: 0.3643 - val_loss: 482975875072.0000 - val_accuracy: 0.4085
Epoch 4/50
9/9 [==============================] - 0s 8ms/step - loss: 155639463936.0000 - accuracy: 0.3714 - val_loss: 327558397952.0000 - val_accuracy: 0.4085
Epoch 5/50
9/9 [==============================] - 0s 7ms/step - loss: 73390284800.0000 - accuracy: 0.3786 - val_loss: 278448308224.0000 - val_accuracy: 0.4085
Epoch 6/50
9/9 [==============================] - 0s 9ms/step - loss: 45789433856.0000 - accuracy: 0.3929 - val_loss: 260313808896.0000 - val_accuracy: 0.4225
Epoch 7/50
9/9 [=====================